# Topik: Analisis Sentimen Opini Publik terhadap Kebijakan Pemindahan Dana Rp 200 Triliun ke Bank Himbara oleh Menteri Keuangan Purbaya melalui Komentar YouTube

## Notebook: Data Scraping dan Filtering

**Anggota Kelompok:**
- 140810230011 - Lukas Austin
- 140810230045 - Devin Suryadi
- 140810230057 - Orlando Bloem Sutono

Mata Kuliah: Data Mining <br>
Kelas: A <br>
Dosen: Bu Helen

---

## 1. Import Library
Import semua library yang diperlukan untuk proses scraping data komentar dari YouTube API.

In [1]:
from dotenv import load_dotenv
import os
import pandas as pd
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
from datetime import datetime
import pandas as pd

print("Library siap digunakan.")

Library siap digunakan.


## 2. Load Youtube API Key
Bagian ini digunakan untuk memuat file `.env` yang berisi API Key YouTube.

In [2]:
load_dotenv()

True

## 3. Inisialisasi API Key
Kode berikut digunakan untuk mengambil nilai API Key dari environment variable dan memverifikasi bahwa kunci berhasil dimuat.


In [3]:
API_KEY = os.getenv("YOUTUBE_API_KEY")

if not API_KEY:
    print("Error: YOUTUBE_API_KEY not found.")
else:
    print("API Key loaded successfully.")

API Key loaded successfully.


## 4. Daftar Video
Bagian ini berisi daftar `video_id` dari video YouTube yang akan digunakan untuk proses scraping komentar.
Setiap video ID merepresentasikan satu video berbeda yang relevan dengan topik.

In [ ]:
# The list of video IDs
VIDEO_ID_LIST = [
    '585-A9uh6Qc',
    'lyBu3Ew3ZL4',
    'xaT4GPEQhD0',
    'S-uR-p--OHs',
    'qO3LAcESAIA',
    'HNr2Zkbi_LA',
    '_3J7dktLIm4',
    'tOUgdSNKHkA',
    'qhePd6FzOaw',
    '0wUxI5rLVaw',
    '_qRzTJIDAHA',
    'GQ_YTznY7gU',
    'JpTV4lXHWOo',
    'BCFQcqMHBKo'
]

# Output file
OUTPUT_CSV_FILE = "dataset/dataset_raw.csv"

# Scraper settings
COMMENTS_PER_VIDEO = 200
SORT_ORDER = "relevance"

## 5. Inisialisasi YouTube API Client
Dilakukan inisialisasi client API untuk mengakses data publik dari YouTube menggunakan library `googleapiclient.discovery`.

In [13]:
client = build("youtube", "v3", developerKey=API_KEY)

## 6. Scraping Komentar
Ini adalah bagian utama yang melakukan proses pengambilan data komentar dari setiap video.  
Setiap komentar yang diperoleh akan disimpan dalam list, kemudian dikonversi menjadi DataFrame.

In [14]:
all_comments_list = []
total_videos = len(VIDEO_ID_LIST)

if client:
    print(f"Memulai scraping untuk {total_videos} video...")
    
    # Loop untuk setiap video ID
    for i, video_id in enumerate(VIDEO_ID_LIST, 1):
        print(f"[{i}/{total_videos}] Memproses ID: {video_id}...")
        
        try:
            count = 0
            next_page_token = None
            
            # Loop untuk pagination hingga mencapai COMMENTS_PER_VIDEO
            while count < COMMENTS_PER_VIDEO:
                # Hitung sisa komentar yang dibutuhkan
                remaining = COMMENTS_PER_VIDEO - count
                max_results = min(100, remaining)  # YouTube API limit: 100 per request
                
                # Meminta 'commentThreads' (komentar top-level)
                request = client.commentThreads().list(
                    part="snippet",
                    videoId=video_id,
                    maxResults=max_results,
                    pageToken=next_page_token,
                    order=SORT_ORDER,
                    textFormat="plainText"
                )
                response = request.execute()
                
                # Loop untuk setiap komentar yang didapat
                for item in response.get("items", []):
                    # Mengambil data dari 'snippet'
                    snippet = item.get('snippet', {}).get('topLevelComment', {}).get('snippet', {})
                    
                    published_at_raw = snippet.get("publishedAt", "")
                    try:
                        published_at = datetime.fromisoformat(published_at_raw.rstrip('Z')).strftime('%Y-%m-%d')
                    except ValueError:
                        published_at = published_at_raw # Gunakan teks asli jika format gagal

                    # Tambahkan data ke list utama
                    all_comments_list.append({
                        "id_comment": item.get('id', ''),
                        "comment": snippet.get('textDisplay', '').replace("\n", " ").replace("\r", " "),
                        "author": snippet.get('authorDisplayName', ''),
                        "like_count": snippet.get('likeCount', 0),
                        "time": published_at,
                        "video_id": video_id
                    })
                    count += 1
                
                # Cek apakah ada halaman berikutnya
                next_page_token = response.get('nextPageToken')
                if not next_page_token:
                    # Tidak ada lagi komentar tersedia
                    break

            print(f"> Total: {count} komentar.")
            
        except HttpError as e:
            print(f"> Gagal mengambil data untuk {video_id}. Error: {e}. Lanjut...")

    print("\nDone scraping")

    df_raw = pd.DataFrame(all_comments_list)

else:
    print("Eksekusi dihentikan karena API client gagal dibuat.")

Memulai scraping untuk 14 video...
[1/14] Memproses ID: 585-A9uh6Qc...
> Total: 200 komentar.
[2/14] Memproses ID: lyBu3Ew3ZL4...
> Total: 200 komentar.
[2/14] Memproses ID: lyBu3Ew3ZL4...
> Total: 200 komentar.
[3/14] Memproses ID: xaT4GPEQhD0...
> Total: 200 komentar.
[3/14] Memproses ID: xaT4GPEQhD0...
> Total: 200 komentar.
[4/14] Memproses ID: S-uR-p--OHs...
> Total: 200 komentar.
[4/14] Memproses ID: S-uR-p--OHs...
> Total: 200 komentar.
[5/14] Memproses ID: qO3LAcESAIA...
> Total: 200 komentar.
[5/14] Memproses ID: qO3LAcESAIA...
> Total: 200 komentar.
[6/14] Memproses ID: HNr2Zkbi_LA...
> Total: 200 komentar.
[6/14] Memproses ID: HNr2Zkbi_LA...
> Total: 200 komentar.
[7/14] Memproses ID: _3J7dktLIm4...
> Total: 200 komentar.
[7/14] Memproses ID: _3J7dktLIm4...
> Total: 200 komentar.
[8/14] Memproses ID: tOUgdSNKHkA...
> Total: 200 komentar.
[8/14] Memproses ID: tOUgdSNKHkA...
> Total: 200 komentar.
[9/14] Memproses ID: qhePd6FzOaw...
> Total: 200 komentar.
[9/14] Memproses ID: 

In [15]:
# Cek dan hapus duplikat berdasarkan id_comment
print(f"Total data sebelum deduplikasi: {len(df_raw)}")
df_raw = df_raw.drop_duplicates(subset=['id_comment'], keep='first')
print(f"Total data setelah deduplikasi: {len(df_raw)}")
print(f"Jumlah duplikat yang dihapus: {len(all_comments_list) - len(df_raw)}")
print("\n" + "=" * 80)
df_raw.head()

Total data sebelum deduplikasi: 2212
Total data setelah deduplikasi: 2212
Jumlah duplikat yang dihapus: 0



,id_comment,comment,author,like_count,time,video_id
0,UgwKOUwkackHqtLfbPR4AaABAg,"Seperti langit dan bumi, kerja pak Purbaya dan...",@bettyutami9973,17,2025-10-13,585-A9uh6Qc
1,UgxeuHpB4tJs3VaxIa94AaABAg,Yg saya heran setiap kali Bang Bennix bilang G...,@misbakulhuda663,89,2025-10-12,585-A9uh6Qc
2,Ugx5BbOJxUVVP_SK1IR4AaABAg,Bubarkan OJK dan pinjol,@ahmadalisudibyo3601,11,2025-10-12,585-A9uh6Qc
3,Ugx_jFD82BwGojRWq4F4AaABAg,"Semoga Purbaya selamat, ngak dikerjain sama ol...",@IndoHog-fe2vd,79,2025-10-12,585-A9uh6Qc
4,Ugwt2EEv2dZFFy9kkql4AaABAg,Ayooo pak pur ttp semangat bangun indonesia 👍,@Emoxn7568,52,2025-10-12,585-A9uh6Qc


## 7. Menyimpan Hasil Scraping Data
Setelah seluruh komentar berhasil diambil, data disimpan ke file CSV agar dapat digunakan pada tahap preprocessing nantinya.

In [16]:
df_raw.to_csv(OUTPUT_CSV_FILE, index=False, encoding='utf-8')
print(f"Berhasil menyimpan df ke csv")

Berhasil menyimpan df ke csv


## 8. Filtering Data
Digunakan untuk melakukan penyaringan awal menggunakan daftar *keyword* yang relevan dengan topik agar komentar tidak relevan tidak masuk ke data untuk diproses.

In [17]:
# keyword filter
keyword_list = [
    "indonesia", "200", "200t", "menteri", "keuangan", "purbaya", "bank", "usaha",
    "kredit", "rakyat", "ekonomi", "negara", "pemerintah", "korup",
    "umkm", "himbara", "suntik", "suntikan", "triliun", "likuiditas", "dana",
    "harga", "bunga", "uang", "pur", "pak men", "mentri", "menkeu", "pak",
    "awas", "duit", "masyarakat", "bapak", "tepat", "sasaran", "dulu", "gaji", "beli",
    "dirut"
]

print(f"Total keyword untuk filter: {len(keyword_list)}")

Total keyword untuk filter: 39


In [18]:
keyword_pattern = '|'.join(keyword_list)
mask_relevan = df_raw['comment'].str.contains(keyword_pattern, case=False, na=False)

# Data relevan
df_filter = df_raw[mask_relevan]
df_not_used = df_raw[~mask_relevan]

print(f"Data relevan: {len(df_filter)}")
print(f"Data tidak relevan: {len(df_not_used)}")

Data relevan: 1763
Data tidak relevan: 449


In [19]:
df_not_used.head()

,id_comment,comment,author,like_count,time,video_id
1,UgxeuHpB4tJs3VaxIa94AaABAg,Yg saya heran setiap kali Bang Bennix bilang G...,@misbakulhuda663,89,2025-10-12,585-A9uh6Qc
2,Ugx5BbOJxUVVP_SK1IR4AaABAg,Bubarkan OJK dan pinjol,@ahmadalisudibyo3601,11,2025-10-12,585-A9uh6Qc
6,UgxdamfgLoDzVgHh8Rl4AaABAg,Kalau dari video yang udah-udah 5k likes aja p...,@roe_goodknight,20,2025-10-12,585-A9uh6Qc
9,Ugyo7Zh2IFIIFiWQVcd4AaABAg,Yoookk otw sejuta subs bang ben,@BuanaPrasetya,1,2025-10-12,585-A9uh6Qc
10,UgxcebDLXCGAQB3wV4l4AaABAg,Bagus banget nih penjelasan Bennix. Jelas dan ...,@alphagate9044,1,2025-10-13,585-A9uh6Qc


In [20]:
df_filter.head()

,id_comment,comment,author,like_count,time,video_id
0,UgwKOUwkackHqtLfbPR4AaABAg,"Seperti langit dan bumi, kerja pak Purbaya dan...",@bettyutami9973,17,2025-10-13,585-A9uh6Qc
3,Ugx_jFD82BwGojRWq4F4AaABAg,"Semoga Purbaya selamat, ngak dikerjain sama ol...",@IndoHog-fe2vd,79,2025-10-12,585-A9uh6Qc
4,Ugwt2EEv2dZFFy9kkql4AaABAg,Ayooo pak pur ttp semangat bangun indonesia 👍,@Emoxn7568,52,2025-10-12,585-A9uh6Qc
5,UgzXTvqGD_Xumg-3Bvp4AaABAg,semoga usaha kecil dan daya beli meningkat. yaa,@echonur7829,2,2025-10-12,585-A9uh6Qc
7,Ugw3_I_8s-uKmhp3spF4AaABAg,"Ah mulai besuk mo pinjem ke bank ah ,bye ito i...",@Herupr2125,3,2025-10-12,585-A9uh6Qc


## 9. Save Hasil Filter
Hasil filtering disimpan dalam file `dataset_filtered.csv`.

In [ ]:
OUTPUT_FILE_FILTER = "dataset/dataset_filtered.csv"
df_filter.to_csv(OUTPUT_FILE_FILTER, index=False, encoding='utf-8')
print(f"Berhasil menyimpan df_filter ke: {OUTPUT_FILE_FILTER}")

Berhasil menyimpan df_filter ke: dataset_filtered.csv


## 10. Kesimpulan
Tahap scraping dan filtering sudah berhasil dilakukan.  
Dataset `dataset_filtered.csv` siap digunakan untuk tahap **preprocessing teks**.